Cleaned and Combined Dataset

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

In [2]:

climate = pd.read_csv('wildfire-temperature-rainfall.csv')
climate = climate.drop(['MeasurementID', 'LocationID', 'UniqueID'], axis=1)
climate = climate[climate['Location'] != 'Imperial']
climate = climate[climate['Location'] != 'San Francisco']
climate_labels = climate['Location']
print(climate_labels.nunique())
climate.head()

56


,AcresBurned,Latitude,Longitude,Month,Year,Location,Value,MinTemp,MaxTemp,AvgTemp
0,38,37.742410,-121.862160,7,2013,Alameda,5.08,48.2,70.8,59.5
1,240,37.775057,-121.737213,6,2013,Alameda,5.08,48.2,70.8,59.5
2,50,37.739300,-121.586900,7,2013,Alameda,5.08,48.2,70.8,59.5
3,96,38.277830,-120.966016,6,2013,Amador,12.25,46.2,72.1,59.1
4,305,39.439000,-121.497000,10,2013,Butte,12.19,46.8,74.1,60.4


In [3]:
features = climate.drop('Location', axis=1)
labels = climate['Location']

In [4]:
dummies = pd.get_dummies(labels)
county_names = dummies.columns
#print(county_names)
one_hot_labels = dummies.to_numpy()
scaler = StandardScaler()
#acres = features['AcresBurned'].to_numpy().reshape(-1,1)
scaled = scaler.fit_transform(features)
one_hot_labels[0]

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=uint8)

In [5]:
scaled

array([[-0.16515557, -0.02341558, -0.1953067 , ...,  0.14943118,
        -0.2566135 , -0.05711901],
       [-0.15809617, -0.02318713, -0.18535571, ...,  0.14943118,
        -0.2566135 , -0.05711901],
       [-0.1647362 , -0.02343734, -0.17338453, ...,  0.14943118,
        -0.2566135 , -0.05711901],
       ...,
       [-0.16403725, -0.01352079, -0.17153126, ...,  0.25889209,
         0.11528025,  0.19821298],
       [-0.16515557, -0.01419082, -0.15452855, ...,  0.25889209,
         0.11528025,  0.19821298],
       [-0.16368777, -0.0128884 , -0.15244592, ...,  0.25889209,
         0.11528025,  0.19821298]])

In [6]:
X_train, X_test, y_train, y_test = train_test_split(scaled, one_hot_labels, test_size=0.2)


In [7]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

Model

In [8]:
import sys
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import tensorflow as tf
from tensorflow.keras import layers, Sequential, Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from tensorflow.keras.layers.experimental import preprocessing

In [9]:

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  512
hidden_nodes_layer2 = 256
hidden_nodes_layer3 = 128
hidden_nodes_layer4 = 64
hidden_nodes_layer5 = 64
hidden_nodes_layer6 = 16
nn = tf.keras.models.Sequential()

# First hidden layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Fourth hidden layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu"))
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer5, activation="relu"))
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer6, activation="relu"))
# Output layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               5120      
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_3 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_4 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_5 (Dense)              (None, 16)                1040      
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 1

In [10]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [11]:
# Compile the model
#  YOUR CODE GOES HERE
nn.compile(loss="mse", optimizer="adam", metrics=["mae","accuracy"])

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    period=5)

In [14]:
# Train the model
#  YOUR CODE GOES HEREfit_model = nn.fit(X_train_scaled,y_train,epochs=100)
fit_model = nn.fit(X_train_scaled,y_train,epochs=500)

Epoch 1/500
37/37 [==============================] - 0s 4ms/step - loss: 0.0175 - mae: 0.0350 - accuracy: 0.9821
Epoch 2/500
37/37 [==============================] - 0s 3ms/step - loss: 0.0175 - mae: 0.0349 - accuracy: 0.9821
Epoch 3/500
37/37 [==============================] - 0s 3ms/step - loss: 0.0175 - mae: 0.0350 - accuracy: 0.9821
Epoch 4/500
37/37 [==============================] - 0s 3ms/step - loss: 0.0175 - mae: 0.0349 - accuracy: 0.9821
Epoch 5/500
37/37 [==============================] - 0s 3ms/step - loss: 0.0175 - mae: 0.0350 - accuracy: 0.9821
Epoch 6/500
37/37 [==============================] - 0s 3ms/step - loss: 0.0175 - mae: 0.0350 - accuracy: 0.9821
Epoch 7/500
37/37 [==============================] - 0s 3ms/step - loss: 0.0175 - mae: 0.0350 - accuracy: 0.9821
Epoch 8/500
37/37 [==============================] - 0s 3ms/step - loss: 0.0175 - mae: 0.0349 - accuracy: 0.9821
Epoch 9/500
37/37 [==============================] - 0s 3ms/step - loss: 0.0175 - mae: 0.0349 - 

37/37 [==============================] - 0s 4ms/step - loss: 0.0175 - mae: 0.0350 - accuracy: 0.9821
Epoch 73/500
37/37 [==============================] - 0s 4ms/step - loss: 0.0175 - mae: 0.0350 - accuracy: 0.9821
Epoch 74/500
37/37 [==============================] - 0s 3ms/step - loss: 0.0175 - mae: 0.0350 - accuracy: 0.9821
Epoch 75/500
37/37 [==============================] - 0s 4ms/step - loss: 0.0175 - mae: 0.0350 - accuracy: 0.9821
Epoch 76/500
37/37 [==============================] - 0s 3ms/step - loss: 0.0175 - mae: 0.0350 - accuracy: 0.9821
Epoch 77/500
37/37 [==============================] - 0s 3ms/step - loss: 0.0175 - mae: 0.0350 - accuracy: 0.9821
Epoch 78/500
37/37 [==============================] - 0s 3ms/step - loss: 0.0175 - mae: 0.0350 - accuracy: 0.9821
Epoch 79/500
37/37 [==============================] - 0s 3ms/step - loss: 0.0175 - mae: 0.0350 - accuracy: 0.9821
Epoch 80/500
37/37 [==============================] - 0s 3ms/step - loss: 0.0175 - mae: 0.0350 - accu

37/37 [==============================] - 0s 3ms/step - loss: 0.0175 - mae: 0.0350 - accuracy: 0.9821
Epoch 144/500
37/37 [==============================] - 0s 3ms/step - loss: 0.0175 - mae: 0.0350 - accuracy: 0.9821
Epoch 145/500
37/37 [==============================] - 0s 3ms/step - loss: 0.0175 - mae: 0.0350 - accuracy: 0.9821
Epoch 146/500
37/37 [==============================] - 0s 3ms/step - loss: 0.0175 - mae: 0.0350 - accuracy: 0.9821
Epoch 147/500
37/37 [==============================] - 0s 3ms/step - loss: 0.0175 - mae: 0.0350 - accuracy: 0.9821
Epoch 148/500
37/37 [==============================] - 0s 3ms/step - loss: 0.0175 - mae: 0.0350 - accuracy: 0.9821
Epoch 149/500
37/37 [==============================] - 0s 3ms/step - loss: 0.0175 - mae: 0.0351 - accuracy: 0.9821
Epoch 150/500
37/37 [==============================] - 0s 3ms/step - loss: 0.0175 - mae: 0.0351 - accuracy: 0.9821
Epoch 151/500
37/37 [==============================] - 0s 3ms/step - loss: 0.0175 - mae: 0.035

37/37 [==============================] - 0s 4ms/step - loss: 0.0175 - mae: 0.0351 - accuracy: 0.9821
Epoch 215/500
37/37 [==============================] - 0s 4ms/step - loss: 0.0175 - mae: 0.0351 - accuracy: 0.9821
Epoch 216/500
37/37 [==============================] - 0s 4ms/step - loss: 0.0175 - mae: 0.0351 - accuracy: 0.9821
Epoch 217/500
37/37 [==============================] - 0s 4ms/step - loss: 0.0175 - mae: 0.0350 - accuracy: 0.9821
Epoch 218/500
37/37 [==============================] - 0s 5ms/step - loss: 0.0175 - mae: 0.0351 - accuracy: 0.9821
Epoch 219/500
37/37 [==============================] - 0s 4ms/step - loss: 0.0175 - mae: 0.0351 - accuracy: 0.9821
Epoch 220/500
37/37 [==============================] - 0s 4ms/step - loss: 0.0175 - mae: 0.0350 - accuracy: 0.9821
Epoch 221/500
37/37 [==============================] - 0s 4ms/step - loss: 0.0175 - mae: 0.0351 - accuracy: 0.9821
Epoch 222/500
37/37 [==============================] - 0s 4ms/step - loss: 0.0175 - mae: 0.035

37/37 [==============================] - 0s 4ms/step - loss: 0.0175 - mae: 0.0351 - accuracy: 0.9821
Epoch 286/500
37/37 [==============================] - 0s 5ms/step - loss: 0.0175 - mae: 0.0351 - accuracy: 0.9821
Epoch 287/500
37/37 [==============================] - 0s 5ms/step - loss: 0.0175 - mae: 0.0351 - accuracy: 0.9821
Epoch 288/500
37/37 [==============================] - 0s 4ms/step - loss: 0.0175 - mae: 0.0351 - accuracy: 0.9821
Epoch 289/500
37/37 [==============================] - 0s 4ms/step - loss: 0.0175 - mae: 0.0351 - accuracy: 0.9821
Epoch 290/500
37/37 [==============================] - 0s 4ms/step - loss: 0.0175 - mae: 0.0351 - accuracy: 0.9821
Epoch 291/500
37/37 [==============================] - 0s 4ms/step - loss: 0.0175 - mae: 0.0351 - accuracy: 0.9821
Epoch 292/500
37/37 [==============================] - 0s 5ms/step - loss: 0.0175 - mae: 0.0351 - accuracy: 0.9821
Epoch 293/500
37/37 [==============================] - 0s 4ms/step - loss: 0.0175 - mae: 0.035

37/37 [==============================] - 0s 5ms/step - loss: 0.0175 - mae: 0.0351 - accuracy: 0.9821
Epoch 357/500
37/37 [==============================] - 0s 4ms/step - loss: 0.0175 - mae: 0.0350 - accuracy: 0.9821
Epoch 358/500
37/37 [==============================] - 0s 4ms/step - loss: 0.0175 - mae: 0.0351 - accuracy: 0.9821
Epoch 359/500
37/37 [==============================] - 0s 4ms/step - loss: 0.0175 - mae: 0.0351 - accuracy: 0.9821
Epoch 360/500
37/37 [==============================] - 0s 4ms/step - loss: 0.0175 - mae: 0.0351 - accuracy: 0.9821
Epoch 361/500
37/37 [==============================] - 0s 5ms/step - loss: 0.0175 - mae: 0.0351 - accuracy: 0.9821
Epoch 362/500
37/37 [==============================] - 0s 4ms/step - loss: 0.0175 - mae: 0.0351 - accuracy: 0.9821
Epoch 363/500
37/37 [==============================] - 0s 4ms/step - loss: 0.0175 - mae: 0.0351 - accuracy: 0.9821
Epoch 364/500
37/37 [==============================] - 0s 4ms/step - loss: 0.0175 - mae: 0.035

37/37 [==============================] - 0s 6ms/step - loss: 0.0175 - mae: 0.0351 - accuracy: 0.9821
Epoch 428/500
37/37 [==============================] - 0s 6ms/step - loss: 0.0175 - mae: 0.0351 - accuracy: 0.9821
Epoch 429/500
37/37 [==============================] - 0s 5ms/step - loss: 0.0175 - mae: 0.0351 - accuracy: 0.9821
Epoch 430/500
37/37 [==============================] - 0s 5ms/step - loss: 0.0175 - mae: 0.0351 - accuracy: 0.9821
Epoch 431/500
37/37 [==============================] - 0s 5ms/step - loss: 0.0175 - mae: 0.0351 - accuracy: 0.9821
Epoch 432/500
37/37 [==============================] - 0s 4ms/step - loss: 0.0175 - mae: 0.0351 - accuracy: 0.9821
Epoch 433/500
37/37 [==============================] - 0s 5ms/step - loss: 0.0175 - mae: 0.0351 - accuracy: 0.9821
Epoch 434/500
37/37 [==============================] - 0s 5ms/step - loss: 0.0175 - mae: 0.0351 - accuracy: 0.9821
Epoch 435/500
37/37 [==============================] - 0s 5ms/step - loss: 0.0175 - mae: 0.035

37/37 [==============================] - 0s 5ms/step - loss: 0.0175 - mae: 0.0351 - accuracy: 0.9821
Epoch 499/500
37/37 [==============================] - 0s 5ms/step - loss: 0.0175 - mae: 0.0351 - accuracy: 0.9821
Epoch 500/500
37/37 [==============================] - 0s 5ms/step - loss: 0.0175 - mae: 0.0351 - accuracy: 0.9821


In [17]:
# Evaluate the model using the test data
model_loss,mae, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, MAE:{mae} Accuracy: {model_accuracy}")

10/10 - 0s - loss: 0.0175 - mae: 0.0350 - accuracy: 0.9821
Loss: 0.017538325861096382, MAE:0.035031240433454514 Accuracy: 0.9821428060531616
